In [1]:
#### This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/datarush-2023/sample_submission.csv
/kaggle/input/datarush-2023/train.csv
/kaggle/input/datarush-2023/test.csv
/kaggle/input/datarush-2023/dataset/dataset/sample_submission.csv
/kaggle/input/datarush-2023/dataset/dataset/train.csv
/kaggle/input/datarush-2023/dataset/dataset/test.csv


In [2]:
df1=pd.read_csv("/kaggle/input/datarush-2023/train.csv")
df2=pd.read_csv("/kaggle/input/datarush-2023/test.csv")


In [3]:
#lets do EDA for train data
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 861236 entries, 0 to 861235
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        861236 non-null  object
 1   title     861236 non-null  object
 2   abstract  861236 non-null  object
 3   category  861232 non-null  object
dtypes: object(4)
memory usage: 26.3+ MB


In [4]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43785 entries, 0 to 43784
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        43785 non-null  object
 1   title     43785 non-null  object
 2   abstract  43785 non-null  object
 3   category  43785 non-null  object
dtypes: object(4)
memory usage: 1.3+ MB


In [5]:
df1['category'].unique()

array(['cs', 'math', 'physics', 'eess', 'cond-mat', 'q-bio', 'stat',
       'hep-ph', 'math-ph', 'hep-th', 'astro-ph', 'gr-qc', 'nlin',
       'quant-ph', 'nucl-th', 'q-fin', 'nucl-ex', 'hep-ex', 'econ',
       'hep-lat', 'funct-an', nan, 'alg-geom', 'q-alg'], dtype=object)

In [6]:
df2['category'].unique()

array(['q-fin', 'cond-mat', 'math', 'cs', 'stat', 'eess', 'astro-ph',
       'gr-qc', 'nucl-th', 'physics', 'quant-ph', 'hep-ph', 'hep-th',
       'hep-ex', 'hep-lat', 'math-ph', 'econ', 'q-bio', 'nucl-ex', 'nlin'],
      dtype=object)

In [7]:
#clean the dataset for null value
#Check for any null value in the target variable
np.sum(df1['category'].isnull())

4

In [8]:
df1.dropna(axis=0, inplace=True)


In [9]:
np.sum(df1['category'].isnull())

0

Data pre-processing
Before training the model, we will perform various pre-processing steps on the dataset such as:


Converting the text document to lowercase for better generalization.
Removing stop words.
Cleaning the punctuation (to reduce unnecessary noise from the dataset).
Removing the repeating characters from the words 
Removing the URLs as they do not have any significant importance.

and much more, we will see this in detail later...

In [10]:
# Making statement text, and abstract in lower case :
df1['title'] = df1['title'].str.lower()
df1['abstract'] = df1['abstract'].str.lower()
df2['title'] = df2['title'].str.lower()
df2['abstract'] = df2['abstract'].str.lower()



In [11]:
   #  Cleaning and removing punctuations :
import string
import re

english_punctuations = string.punctuation
punctuations_list = english_punctuations

def cleaning_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)


df1['title'] = df1['title'].apply(lambda x: cleaning_punctuations(x))
df1['abstract'] = df1['abstract'].apply(lambda x: cleaning_punctuations(x))
df2['title'] = df2['title'].apply(lambda x: cleaning_punctuations(x))
df2['abstract'] = df2['abstract'].apply(lambda x: cleaning_punctuations(x))


In [12]:
# Cleaning and removing repeating characters :
def cleaning_repeating_char(text):
    return re.sub(r'(.)1+', r'1', text)


df1['title'] = df1['title'].apply(lambda x: cleaning_repeating_char(x))
df1['abstract'] = df1['abstract'].apply(lambda x: cleaning_repeating_char(x))
df2['title'] = df2['title'].apply(lambda x: cleaning_repeating_char(x))
df2['abstract'] = df2['abstract'].apply(lambda x: cleaning_repeating_char(x))


In [13]:
# Cleaning and removing Numeric numbers :
def cleaning_numbers(data):
    return re.sub('[0-9]+', '', data)

df1['title'] = df1['title'].apply(lambda x: cleaning_numbers(x))
df1['abstract'] = df1['abstract'].apply(lambda x: cleaning_numbers(x))
df2['title'] = df2['title'].apply(lambda x: cleaning_numbers(x))
df2['abstract'] = df2['abstract'].apply(lambda x: cleaning_numbers(x))


In [14]:
df1=df1.head(100000)

In [15]:
# Getting tokenization of text in title and abstract  :
from nltk.tokenize import word_tokenize

df1['title'] = df1['title'].apply(word_tokenize)
df1['abstract'] = df1['abstract'].apply(word_tokenize)
df2['title'] = df2['title'].apply(word_tokenize)
df2['abstract'] = df2['abstract'].apply(word_tokenize)



In [16]:

df2['title'] = df1['title'].apply(lambda x: stemming_on_text(x))
df2['abstract'] = df2['abstract'].apply(lambda x: stemming_on_text(x))



In [17]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(df1.category)
y2 = le.transform(df2.category)



In [18]:
df1['text']=df1['title']+df1['abstract']

In [19]:
df2['text']=df2['title']+df2['abstract']

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(ngram_range=(1,2),vocabulary=None,max_features=None)
text = [' '.join(sentence) for sentence in df1.text]
X_tfidf_title=tfidf_vect.fit_transform(text)

text2 = [' '.join(sentence) for sentence in df2.text]
X_tfidf_title2=tfidf_vect.transform(text2)





In [21]:
X_tfidf_title2.shape

(43785, 2710007)

In [22]:
from sklearn.metrics import f1_score
def print_score(y_pred, clf):
    print("Clf: ", clf.__class__.__name__)
    print("F1 score: {}".format(f1_score(y_pred,y2,zero_division='warn',average="weighted")))
    y_pred=le.inverse_transform(y_pred)
    print(y_pred)
    print("---")    

In [23]:
from sklearn.linear_model import SGDClassifier

from sklearn.multiclass import OneVsRestClassifier
sgd = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42, max_iter=6, tol=None)

for classifier in [ sgd]:
    clf = OneVsRestClassifier(classifier)
    clf.fit(X_tfidf_title, y)
    y_pred = clf.predict(X_tfidf_title2)
    print_score(y_pred, classifier)
 

    
    
    


Clf:  SGDClassifier
F1 score: 0.6805258339266208
['cs' 'cond-mat' 'cs' ... 'cs' 'cs' 'cs']
---


In [24]:


# Create a dataframe with the test data and the predictions
submission = pd.DataFrame({'Id': df2['id'], 'Prediction': y_pred})

# Write the dataframe to a CSV file
submission.to_csv('submission.csv', index=False)
